## Imports

In [10]:
import unittest
import os
import sys
import pathlib
import urllib
import shutil
import re
import zipfile

import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Model

In [ ]:
class WaveletTransformer(nn.Module):
    def __init__(self):
        super(WaveletTransformer, self).__init__()
        pass

In [60]:
class ConvNet(nn.Module):
    def __init__(self, in_channels=3):
        super(ConvNet, self).__init__()
                
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 10, kernel_size=(3,21)),
            nn.ReLU(),
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(3,21)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(4,21)),
            nn.ReLU(),
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(4,21)),
            nn.ReLU(),
        )
        
        self.fc = nn.Linear(1800, 50)
        
    def forward(self, x):
        print(x.shape)
        out = self.layer1(x)
        print(out.shape)
        out = self.layer2(out)
        print(out.shape)

        # out = out.reshape(out.size(0), -1)

        out = self.layer3(out)
        print(out.shape)
        out = self.layer4(out)
        
        print(out.shape)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        print(out.shape)
        return out

In [ ]:
class BRNN(nn.Module):
    def __init__(self):
        super(BRNN, self).__init__()
        self.bi_grus = torch.nn.GRU(input_size=20, hidden_size=50, num_layers=1, batch_first=False, bidirectional=True)
        
    def forward(self, X):
        pass

In [61]:
display(ConvNet(in_channels=3))

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Linear(in_features=1800, out_features=50, bias=True)
)

In [63]:
in_channels = 3
x0 = torch.rand((3, 20, 300)).unsqueeze(0)
encoder_cnn = ConvNet(in_channels)
print(encoder_cnn)

h = encoder_cnn(x0)
# print(h.shape)

test.assertEqual(h.dim(), 2)
test.assertSequenceEqual(h.shape, (1, 50))

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Linear(in_features=1800, out_features=50, bias=True)
)
torch.Size([1, 3, 20, 300])
torch.Size([1, 10, 18, 280])
torch.Size([1, 10, 8, 130])
torch.Size([1, 10, 5, 110])
torch.Size([1, 10, 2, 90])
torch.Size([1, 50])


# Training 

In [66]:
model = ConvNet()

learning_rate = 0.001
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

NameError: name 'train_loader' is not defined